# 🔋 LSTM Complet Interface Entraînement - Système de Prédiction Énergétique

## 🎯 Objectif
Ce notebook permet à l'utilisateur de choisir entre :
1. **Utiliser les modèles pré-entraînés** de la région Benten 
2. **Ré-entraîner de nouveaux modèles** sur ses propres données

Le notebook combine les fonctionnalités de "LSTM complet" et "LSTM Generation" sans optimisation Optuna pour réduire le temps d'exécution.

## 📋 Structure du Notebook
1. **Configuration et choix utilisateur** - Sélection du mode d'utilisation
2. **Chargement et préparation des données** - Import et preprocessing
3. **Modèles pré-entraînés OU Entraînement** - Selon le choix utilisateur
4. **Prédictions et évaluation** - Génération des résultats
5. **Sauvegarde et export** - Enregistrement des modèles et résultats

---

## 📚 1. Importation des Bibliothèques

In [ ]:
# Importation des bibliothèques essentielles
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Preprocessing et métriques
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Utilitaires
import os
import pickle
import joblib
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output

print("✅ Toutes les bibliothèques ont été importées avec succès!")
print(f"🔧 Version TensorFlow: {tf.__version__}")
print(f"📊 Version Pandas: {pd.__version__}")
print(f"🔢 Version NumPy: {np.__version__}")

In [ ]:
# Configuration des variables d'environnement
import os

# Variables configurables depuis l'interface Streamlit
LSTM_MODE = os.environ.get('LSTM_MODE', 'pretrained')  # 'pretrained' ou 'retrain'
DATA_FILE = os.environ.get('DATA_FILE', 'data.csv')
STREAMLIT_MODE = os.environ.get('STREAMLIT_MODE', 'false').lower() == 'true'

print(f"🔧 Configuration d'environnement :")
print(f"   • Mode LSTM : {LSTM_MODE}")
print(f"   • Fichier de données : {DATA_FILE}")
print(f"   • Exécution Streamlit : {STREAMLIT_MODE}")

# Configuration pour compatibilité avec l'interface
if STREAMLIT_MODE:
    print("🖥️ Mode interface Streamlit activé - Configuration automatique")

## ⚙️ 2. Configuration et Choix Utilisateur

Choisissez le mode d'utilisation :
- **Mode 1** : Utiliser les modèles pré-entraînés de la région Benten
- **Mode 2** : Ré-entraîner de nouveaux modèles sur vos données

In [ ]:
# Configuration des chemins
DATA_PATH = "../Data/"
MODELS_PATH = "models/"
SCALERS_PATH = "scalers/"

# Créer les dossiers s'ils n'existent pas
os.makedirs(MODELS_PATH, exist_ok=True)
os.makedirs(SCALERS_PATH, exist_ok=True)

print("🔋 Système de Prédiction Énergétique LSTM")
print("=" * 50)

# Variables globales pour stocker les choix
selected_mode = None
selected_data_file = None

if STREAMLIT_MODE:
    # En mode Streamlit, utiliser les variables d'environnement
    selected_mode = LSTM_MODE
    selected_data_file = DATA_FILE
    print("🔧 Configuration automatique (mode Streamlit):")
    print(f"📊 Mode sélectionné: {selected_mode}")
    print(f"📁 Fichier de données: {selected_data_file}")
    
    if selected_mode == 'pretrained':
        print("🎯 Vous allez utiliser les modèles pré-entraînés de la région Benten")
    else:
        print("🔄 Vous allez ré-entraîner de nouveaux modèles")
        
else:
    # Mode interactif avec widgets
    # Interface de choix utilisateur
    mode_selection = widgets.RadioButtons(
        options=[
            ('🎯 Utiliser les modèles pré-entraînés (Région Benten)', 'pretrained'),
            ('🔄 Ré-entraîner de nouveaux modèles', 'retrain')
        ],
        value='pretrained',
        description='Mode:',
        disabled=False,
        style={'description_width': 'initial'}
    )

    # Widget de sélection du fichier de données
    data_file_selection = widgets.Dropdown(
        options=['data.csv', 'train_data.csv', 'test_data.csv', 'uploaded_test_data.csv'],
        value='data.csv',
        description='Fichier de données:',
        disabled=False,
        style={'description_width': 'initial'}
    )

    # Bouton pour confirmer la sélection
    confirm_button = widgets.Button(
        description="✅ Confirmer la sélection",
        button_style='success',
        tooltip='Cliquez pour confirmer votre choix'
    )

    def on_confirm_clicked(b):
        global selected_mode, selected_data_file
        selected_mode = mode_selection.value
        selected_data_file = data_file_selection.value
        
        clear_output()
        print("✅ Configuration confirmée!")
        print(f"📊 Mode sélectionné: {selected_mode}")
        print(f"📁 Fichier de données: {selected_data_file}")
        
        if selected_mode == 'pretrained':
            print("🎯 Vous allez utiliser les modèles pré-entraînés de la région Benten")
        else:
            print("🔄 Vous allez ré-entraîner de nouveaux modèles")

    confirm_button.on_click(on_confirm_clicked)

    # Affichage de l'interface
    print("🔧 Configuration manuelle (mode interactif):")
    display(mode_selection)
    display(data_file_selection)
    display(confirm_button)

## 📊 3. Chargement et Préparation des Données

In [ ]:
# Fonction de chargement des données
def load_and_prepare_data(file_name):
    """Charge et prépare les données pour l'entraînement LSTM"""
    try:
        # Chargement des données
        data_path = os.path.join(DATA_PATH, file_name)
        data = pd.read_csv(data_path)
        
        print(f"✅ Données chargées depuis: {data_path}")
        print(f"📊 Forme des données: {data.shape}")
        print(f"📋 Colonnes disponibles: {list(data.columns)}")
        
        # Vérification de la colonne date
        if 'date' in data.columns:
            data['date'] = pd.to_datetime(data['date'])
            data = data.sort_values('date')
            print("📅 Colonne date formatée et données triées")
        
        # Identification des colonnes de prédiction
        prediction_columns = [col for col in data.columns if col != 'date']
        print(f"🎯 Colonnes de prédiction: {prediction_columns}")
        
        # Vérification des valeurs manquantes
        missing_values = data.isnull().sum()
        if missing_values.sum() > 0:
            print("⚠️ Valeurs manquantes détectées:")
            print(missing_values[missing_values > 0])
            data = data.fillna(method='ffill').fillna(method='bfill')
            print("✅ Valeurs manquantes corrigées")
        
        return data, prediction_columns
        
    except Exception as e:
        print(f"❌ Erreur lors du chargement des données: {str(e)}")
        return None, None

# Déterminer le fichier de données à utiliser
if STREAMLIT_MODE:
    # En mode Streamlit, utiliser le fichier téléversé
    data_file_to_use = selected_data_file if 'selected_data_file' in globals() else DATA_FILE
    print(f"🔧 Mode Streamlit: Utilisation du fichier {data_file_to_use}")
else:
    # En mode interactif, utiliser la sélection utilisateur ou fichier par défaut
    data_file_to_use = selected_data_file if 'selected_data_file' in globals() and selected_data_file else 'data.csv'
    print(f"🔧 Mode interactif: Utilisation du fichier {data_file_to_use}")

# Chargement des données
data, prediction_columns = load_and_prepare_data(data_file_to_use)

if data is not None:
    print("\n📈 Aperçu des données:")
    display(data.head())
    
    print("\n📊 Statistiques descriptives:")
    display(data.describe())
    
    # Visualisation des données (seulement en mode non-Streamlit pour éviter les conflits)
    if not STREAMLIT_MODE:
        plt.figure(figsize=(15, 10))
        
        # Graphique temporel si colonne date disponible
        if 'date' in data.columns:
            for i, col in enumerate(prediction_columns[:4]):  # Limite à 4 colonnes pour la lisibilité
                plt.subplot(2, 2, i+1)
                plt.plot(data['date'], data[col])
                plt.title(f'Évolution de {col}')
                plt.xlabel('Date')
                plt.ylabel(col)
                plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.show()
    else:
        print("📊 Visualisations désactivées en mode Streamlit")
    
else:
    print("❌ Impossible de charger les données. Vérifiez le nom du fichier.")
    if STREAMLIT_MODE:
        raise FileNotFoundError(f"Fichier de données non trouvé: {data_file_to_use}")

## 🎯 4. Choix entre Modèles Pré-entraînés et Ré-entraînement

Cette section exécute différents processus selon votre choix :
- **Modèles pré-entraînés** : Chargement des modèles de la région Benten
- **Ré-entraînement** : Création et entraînement de nouveaux modèles

In [ ]:
# Fonctions utilitaires pour le preprocessing
def create_sequences(data, target_col, sequence_length=60):
    """Crée des séquences pour l'entraînement LSTM"""
    X, y = [], []
    for i in range(sequence_length, len(data)):
        X.append(data[i-sequence_length:i])
        y.append(data[i, target_col])
    return np.array(X), np.array(y)

def prepare_data_for_training(data, target_columns, sequence_length=60, test_size=0.2):
    """Prépare les données pour l'entraînement"""
    results = {}
    
    for target_col in target_columns:
        print(f"\n🎯 Préparation des données pour: {target_col}")
        
        # Sélection des features (toutes les colonnes numériques)
        feature_columns = [col for col in data.columns if col != 'date' and data[col].dtype in ['float64', 'int64']]
        features_data = data[feature_columns].values
        
        # Normalisation
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(features_data)
        
        # Index de la colonne cible
        target_idx = feature_columns.index(target_col)
        
        # Création des séquences
        X, y = create_sequences(scaled_data, target_idx, sequence_length)
        
        # Division train/test
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42, shuffle=False
        )
        
        results[target_col] = {
            'X_train': X_train,
            'X_test': X_test,
            'y_train': y_train,
            'y_test': y_test,
            'scaler': scaler,
            'feature_columns': feature_columns
        }
        
        print(f"✅ Données préparées - Train: {X_train.shape}, Test: {X_test.shape}")
    
    return results

# Fonction pour créer un modèle LSTM
def create_lstm_model(input_shape, neurons=[50, 50], dropout_rate=0.2):
    """Crée un modèle LSTM avec architecture simple"""
    model = Sequential()
    
    # Première couche LSTM
    model.add(LSTM(neurons[0], return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    
    # Deuxième couche LSTM
    model.add(LSTM(neurons[1], return_sequences=False))
    model.add(Dropout(dropout_rate))
    
    # Couche de sortie
    model.add(Dense(1))
    
    # Compilation
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    
    return model

# Vérification des variables d'environnement pour compatibilité Streamlit
if STREAMLIT_MODE:
    # En mode Streamlit, utiliser les variables d'environnement
    selected_mode = LSTM_MODE
    selected_data_file = DATA_FILE
    print(f"🔧 Mode Streamlit détecté - Configuration automatique:")
    print(f"   • Mode: {selected_mode}")
    print(f"   • Fichier: {selected_data_file}")

# Exécution selon le mode choisi
if 'selected_mode' in globals() and selected_mode and data is not None:
    
    if selected_mode == 'pretrained':
        print("🎯 Mode: Utilisation des modèles pré-entraînés")
        print("=" * 50)
        
        # Vérification de l'existence des modèles pré-entraînés
        pretrained_models = {}
        available_models = []
        
        for col in prediction_columns:
            model_path = os.path.join(MODELS_PATH, f"{col}_LSTM.h5")
            scaler_path = os.path.join(SCALERS_PATH, f"{col}_scaler.pkl")
            
            if os.path.exists(model_path) and os.path.exists(scaler_path):
                try:
                    model = load_model(model_path)
                    with open(scaler_path, 'rb') as f:
                        scaler = pickle.load(f)
                    
                    pretrained_models[col] = {'model': model, 'scaler': scaler}
                    available_models.append(col)
                    print(f"✅ Modèle pré-entraîné chargé pour: {col}")
                except Exception as e:
                    print(f"❌ Erreur lors du chargement du modèle {col}: {str(e)}")
            else:
                print(f"⚠️ Modèle pré-entraîné non trouvé pour: {col}")
        
        if available_models:
            print(f"\n🎉 {len(available_models)} modèles pré-entraînés disponibles!")
            models_ready = True
        else:
            print("❌ Aucun modèle pré-entraîné trouvé. Passez au mode ré-entraînement.")
            models_ready = False
    
    else:  # Mode ré-entraînement
        print("🔄 Mode: Ré-entraînement de nouveaux modèles")
        print("=" * 50)
        
        # Préparation des données pour l'entraînement
        training_data = prepare_data_for_training(data, prediction_columns)
        trained_models = {}
        
        for target_col in prediction_columns:
            print(f"\n🚀 Entraînement du modèle pour: {target_col}")
            
            # Récupération des données préparées
            data_dict = training_data[target_col]
            X_train, y_train = data_dict['X_train'], data_dict['y_train']
            X_test, y_test = data_dict['X_test'], data_dict['y_test']
            
            # Création du modèle
            input_shape = (X_train.shape[1], X_train.shape[2])
            model = create_lstm_model(input_shape)
            
            print(f"📊 Architecture du modèle: {model.summary()}")
            
            # Callbacks
            early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
            
            # Entraînement
            print("🔄 Début de l'entraînement...")
            history = model.fit(
                X_train, y_train,
                epochs=50,  # Nombre d'époques réduit pour économiser du temps
                batch_size=32,
                validation_data=(X_test, y_test),
                callbacks=[early_stopping],
                verbose=1
            )
            
            # Sauvegarde du modèle et du scaler
            model_path = os.path.join(MODELS_PATH, f"{target_col}_LSTM.h5")
            scaler_path = os.path.join(SCALERS_PATH, f"{target_col}_scaler.pkl")
            
            model.save(model_path)
            with open(scaler_path, 'wb') as f:
                pickle.dump(data_dict['scaler'], f)
            
            trained_models[target_col] = {
                'model': model,
                'scaler': data_dict['scaler'],
                'history': history,
                'test_data': (X_test, y_test)
            }
            
            print(f"✅ Modèle entraîné et sauvegardé pour: {target_col}")
            
            # Affichage de la courbe d'apprentissage
            plt.figure(figsize=(12, 4))
            plt.subplot(1, 2, 1)
            plt.plot(history.history['loss'], label='Train Loss')
            plt.plot(history.history['val_loss'], label='Validation Loss')
            plt.title(f'Courbe d\'apprentissage - {target_col}')
            plt.xlabel('Époque')
            plt.ylabel('Loss')
            plt.legend()
            
            plt.subplot(1, 2, 2)
            plt.plot(history.history['mae'], label='Train MAE')
            plt.plot(history.history['val_mae'], label='Validation MAE')
            plt.title(f'Erreur absolue moyenne - {target_col}')
            plt.xlabel('Époque')
            plt.ylabel('MAE')
            plt.legend()
            
            plt.tight_layout()
            plt.show()
        
        print(f"\n🎉 Entraînement terminé pour {len(trained_models)} modèles!")
        models_ready = True

else:
    print("⚠️ Veuillez d'abord confirmer votre sélection et charger les données.")

## 🔮 5. Génération des Prédictions

Cette section utilise les modèles (pré-entraînés ou nouvellement entraînés) pour générer des prédictions sur les données.

In [ ]:
# Fonction pour générer des prédictions
def generate_predictions(models_dict, data, sequence_length=60, prediction_days=30):
    """Génère des prédictions avec les modèles disponibles"""
    predictions_results = {}
    
    for target_col, model_info in models_dict.items():
        print(f"\n🔮 Génération des prédictions pour: {target_col}")
        
        model = model_info['model']
        scaler = model_info['scaler']
        
        # Préparation des données pour prédiction
        feature_columns = [col for col in data.columns if col != 'date' and data[col].dtype in ['float64', 'int64']]
        features_data = data[feature_columns].values
        
        # Normalisation avec le scaler du modèle
        scaled_data = scaler.transform(features_data)
        
        # Index de la colonne cible
        target_idx = feature_columns.index(target_col)
        
        # Prendre les dernières séquences pour commencer les prédictions
        last_sequence = scaled_data[-sequence_length:]
        
        # Génération des prédictions futures
        future_predictions = []
        current_sequence = last_sequence.copy()
        
        for _ in range(prediction_days):
            # Reshape pour le modèle LSTM
            input_seq = current_sequence.reshape(1, sequence_length, len(feature_columns))
            
            # Prédiction
            pred = model.predict(input_seq, verbose=0)[0, 0]
            future_predictions.append(pred)
            
            # Mettre à jour la séquence pour la prochaine prédiction
            new_row = current_sequence[-1].copy()
            new_row[target_idx] = pred
            
            # Décaler la séquence
            current_sequence = np.vstack([current_sequence[1:], new_row])
        
        # Dénormalisation des prédictions
        # Créer un array temporaire pour la dénormalisation
        temp_array = np.zeros((len(future_predictions), len(feature_columns)))
        temp_array[:, target_idx] = future_predictions
        denormalized = scaler.inverse_transform(temp_array)
        final_predictions = denormalized[:, target_idx]
        
        # Prédictions sur les données historiques pour évaluation
        historical_predictions = []
        for i in range(sequence_length, len(scaled_data)):
            input_seq = scaled_data[i-sequence_length:i].reshape(1, sequence_length, len(feature_columns))
            pred = model.predict(input_seq, verbose=0)[0, 0]
            historical_predictions.append(pred)
        
        # Dénormalisation des prédictions historiques
        temp_hist = np.zeros((len(historical_predictions), len(feature_columns)))
        temp_hist[:, target_idx] = historical_predictions
        denorm_hist = scaler.inverse_transform(temp_hist)
        final_hist_predictions = denorm_hist[:, target_idx]
        
        # Stockage des résultats
        predictions_results[target_col] = {
            'future_predictions': final_predictions,
            'historical_predictions': final_hist_predictions,
            'actual_values': data[target_col].values[sequence_length:],
            'feature_columns': feature_columns,
            'dates': data['date'].values if 'date' in data.columns else None
        }
        
        print(f"✅ {len(final_predictions)} prédictions futures générées")
        print(f"📊 {len(final_hist_predictions)} prédictions historiques générées")
    
    return predictions_results

# Génération des prédictions selon le mode
if 'models_ready' in globals() and models_ready:
    print("🔮 Génération des prédictions en cours...")
    
    # Sélection des modèles selon le mode
    if selected_mode == 'pretrained':
        active_models = pretrained_models
        print("📊 Utilisation des modèles pré-entraînés")
    else:
        active_models = trained_models
        print("📊 Utilisation des modèles nouvellement entraînés")
    
    # Paramètres de prédiction
    SEQUENCE_LENGTH = 60
    PREDICTION_DAYS = 30
    
    print(f"⚙️ Paramètres: Séquence={SEQUENCE_LENGTH}, Prédictions futures={PREDICTION_DAYS} jours")
    
    # Génération des prédictions
    all_predictions = generate_predictions(
        active_models, 
        data, 
        sequence_length=SEQUENCE_LENGTH, 
        prediction_days=PREDICTION_DAYS
    )
    
    print(f"\n🎉 Prédictions générées pour {len(all_predictions)} variables!")
    
    # Affichage des résultats pour chaque variable
    for target_col, results in all_predictions.items():
        print(f"\n📈 Résultats pour {target_col}:")
        print(f"   • Prédictions futures: {len(results['future_predictions'])} valeurs")
        print(f"   • Prédictions historiques: {len(results['historical_predictions'])} valeurs")
        print(f"   • Valeurs min/max futures: {results['future_predictions'].min():.2f} / {results['future_predictions'].max():.2f}")

else:
    print("⚠️ Les modèles ne sont pas prêts. Exécutez d'abord les cellules précédentes.")

## 📊 6. Évaluation des Modèles

Cette section évalue les performances des modèles en calculant différentes métriques et en générant des visualisations.

In [ ]:
# Fonction d'évaluation des modèles
def evaluate_model_performance(predictions_results):
    """Évalue les performances des modèles avec différentes métriques"""
    evaluation_results = {}
    
    print("📊 Évaluation des performances des modèles")
    print("=" * 50)
    
    for target_col, results in predictions_results.items():
        print(f"\n🎯 Évaluation pour: {target_col}")
        
        actual = results['actual_values']
        predicted = results['historical_predictions']
        
        # Calcul des métriques
        mse = mean_squared_error(actual, predicted)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)
        
        # MAPE (Mean Absolute Percentage Error)
        mape = np.mean(np.abs((actual - predicted) / actual)) * 100
        
        # Stockage des métriques
        metrics = {
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R²': r2,
            'MAPE': mape
        }
        
        evaluation_results[target_col] = metrics
        
        # Affichage des métriques
        print(f"   📈 MSE (Mean Squared Error): {mse:.4f}")
        print(f"   📈 RMSE (Root Mean Squared Error): {rmse:.4f}")
        print(f"   📈 MAE (Mean Absolute Error): {mae:.4f}")
        print(f"   📈 R² (Coefficient de détermination): {r2:.4f}")
        print(f"   📈 MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
        
        # Interprétation du R²
        if r2 >= 0.9:
            print("   ✅ Excellent modèle (R² ≥ 0.9)")
        elif r2 >= 0.8:
            print("   🟢 Bon modèle (R² ≥ 0.8)")
        elif r2 >= 0.6:
            print("   🟡 Modèle acceptable (R² ≥ 0.6)")
        else:
            print("   🔴 Modèle à améliorer (R² < 0.6)")
    
    return evaluation_results

# Fonction de visualisation des résultats
def visualize_predictions(predictions_results, max_points=1000):
    """Visualise les prédictions vs valeurs réelles"""
    n_models = len(predictions_results)
    
    # Calcul du nombre de lignes et colonnes pour les sous-graphiques
    cols = min(2, n_models)
    rows = (n_models + cols - 1) // cols
    
    plt.figure(figsize=(15, 5 * rows))
    
    for idx, (target_col, results) in enumerate(predictions_results.items()):
        # Graphique 1: Comparaison prédictions vs réalité
        plt.subplot(rows, cols, idx + 1)
        
        actual = results['actual_values']
        predicted = results['historical_predictions']
        
        # Limitation du nombre de points pour la lisibilité
        if len(actual) > max_points:
            step = len(actual) // max_points
            actual_plot = actual[::step]
            predicted_plot = predicted[::step]
        else:
            actual_plot = actual
            predicted_plot = predicted
        
        plt.plot(actual_plot, label='Valeurs réelles', alpha=0.7, linewidth=1)
        plt.plot(predicted_plot, label='Prédictions', alpha=0.7, linewidth=1)
        plt.title(f'Prédictions vs Réalité - {target_col}')
        plt.xlabel('Échantillons')
        plt.ylabel('Valeurs')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Graphique des prédictions futures
    plt.figure(figsize=(15, 5 * rows))
    
    for idx, (target_col, results) in enumerate(predictions_results.items()):
        plt.subplot(rows, cols, idx + 1)
        
        future_pred = results['future_predictions']
        
        # Affichage des dernières valeurs historiques pour contexte
        if len(results['actual_values']) > 0:
            last_actual = results['actual_values'][-30:]  # Derniers 30 points
            x_hist = range(-len(last_actual), 0)
            plt.plot(x_hist, last_actual, label='Historique récent', color='blue', alpha=0.7)
        
        # Prédictions futures
        x_future = range(0, len(future_pred))
        plt.plot(x_future, future_pred, label='Prédictions futures', color='red', marker='o', markersize=3)
        
        plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5, label='Début des prédictions')
        plt.title(f'Prédictions futures - {target_col}')
        plt.xlabel('Jours')
        plt.ylabel('Valeurs')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Exécution de l'évaluation
if 'all_predictions' in globals() and all_predictions:
    print("📊 Début de l'évaluation des modèles...")
    
    # Évaluation des performances
    model_metrics = evaluate_model_performance(all_predictions)
    
    # Création d'un DataFrame récapitulatif
    metrics_df = pd.DataFrame(model_metrics).T
    print("\n📋 Récapitulatif des métriques:")
    display(metrics_df.round(4))
    
    # Visualisation des résultats
    print("\n📈 Génération des graphiques...")
    visualize_predictions(all_predictions)
    
    # Graphique en barres des métriques R²
    plt.figure(figsize=(12, 6))
    r2_values = [metrics['R²'] for metrics in model_metrics.values()]
    model_names = list(model_metrics.keys())
    
    colors = ['green' if r2 >= 0.8 else 'orange' if r2 >= 0.6 else 'red' for r2 in r2_values]
    
    plt.bar(model_names, r2_values, color=colors, alpha=0.7)
    plt.title('Coefficient de détermination (R²) par modèle')
    plt.ylabel('R²')
    plt.xlabel('Variables')
    plt.xticks(rotation=45, ha='right')
    plt.ylim(0, 1)
    plt.grid(True, alpha=0.3)
    
    # Ligne de référence pour un bon modèle
    plt.axhline(y=0.8, color='red', linestyle='--', alpha=0.5, label='Seuil bon modèle (0.8)')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Évaluation terminée!")

else:
    print("⚠️ Aucune prédiction trouvée. Exécutez d'abord la section de génération des prédictions.")

## 💾 7. Sauvegarde et Export des Résultats

Cette section sauvegarde les modèles, les métriques et exporte les résultats dans différents formats.

In [ ]:
# Fonction de sauvegarde des résultats
def save_results(predictions_results, model_metrics, output_dir="results"):
    """Sauvegarde les résultats dans différents formats"""
    
    # Créer le dossier de résultats
    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    print(f"💾 Sauvegarde des résultats dans: {output_dir}/")
    
    # 1. Sauvegarde des prédictions en CSV
    for target_col, results in predictions_results.items():
        # Prédictions futures
        future_df = pd.DataFrame({
            'jour': range(1, len(results['future_predictions']) + 1),
            'prediction': results['future_predictions']
        })
        future_file = os.path.join(output_dir, f"{target_col}_predictions_futures_{timestamp}.csv")
        future_df.to_csv(future_file, index=False)
        print(f"   ✅ Prédictions futures sauvées: {future_file}")
        
        # Prédictions historiques vs réalité
        historical_df = pd.DataFrame({
            'valeur_reelle': results['actual_values'],
            'prediction': results['historical_predictions'],
            'erreur': results['actual_values'] - results['historical_predictions']
        })
        historical_file = os.path.join(output_dir, f"{target_col}_evaluation_{timestamp}.csv")
        historical_df.to_csv(historical_file, index=False)
        print(f"   ✅ Évaluation historique sauvée: {historical_file}")
    
    # 2. Sauvegarde des métriques
    metrics_df = pd.DataFrame(model_metrics).T
    metrics_file = os.path.join(output_dir, f"metriques_modeles_{timestamp}.csv")
    metrics_df.to_csv(metrics_file)
    print(f"   ✅ Métriques sauvées: {metrics_file}")
    
    # 3. Rapport de synthèse
    report_file = os.path.join(output_dir, f"rapport_synthese_{timestamp}.txt")
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write("🔋 RAPPORT DE SYNTHÈSE - SYSTÈME DE PRÉDICTION ÉNERGÉTIQUE\\n")
        f.write("=" * 60 + "\\n\\n")
        f.write(f"📅 Date de génération: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n")
        f.write(f"📊 Mode utilisé: {selected_mode}\\n")
        f.write(f"📁 Fichier de données: {selected_data_file}\\n")
        f.write(f"🎯 Nombre de variables prédites: {len(predictions_results)}\\n\\n")
        
        f.write("📈 RÉSUMÉ DES PERFORMANCES:\\n")
        f.write("-" * 30 + "\\n")
        for target_col, metrics in model_metrics.items():
            f.write(f"\\n🔸 {target_col}:\\n")
            f.write(f"   • R² (coefficient de détermination): {metrics['R²']:.4f}\\n")
            f.write(f"   • RMSE (erreur quadratique moyenne): {metrics['RMSE']:.4f}\\n")
            f.write(f"   • MAE (erreur absolue moyenne): {metrics['MAE']:.4f}\\n")
            f.write(f"   • MAPE (erreur absolue en pourcentage): {metrics['MAPE']:.2f}%\\n")
            
            # Évaluation qualitative
            if metrics['R²'] >= 0.9:
                f.write(f"   • Évaluation: Excellent modèle ✅\\n")
            elif metrics['R²'] >= 0.8:
                f.write(f"   • Évaluation: Bon modèle 🟢\\n")
            elif metrics['R²'] >= 0.6:
                f.write(f"   • Évaluation: Modèle acceptable 🟡\\n")
            else:
                f.write(f"   • Évaluation: Modèle à améliorer 🔴\\n")
        
        f.write(f"\\n\\n📊 PRÉDICTIONS GÉNÉRÉES:\\n")
        f.write("-" * 25 + "\\n")
        for target_col, results in predictions_results.items():
            f.write(f"\\n🔸 {target_col}:\\n")
            f.write(f"   • Prédictions futures: {len(results['future_predictions'])} jours\\n")
            f.write(f"   • Valeur min prédite: {results['future_predictions'].min():.2f}\\n")
            f.write(f"   • Valeur max prédite: {results['future_predictions'].max():.2f}\\n")
            f.write(f"   • Valeur moyenne prédite: {results['future_predictions'].mean():.2f}\\n")
    
    print(f"   ✅ Rapport de synthèse sauvé: {report_file}")
    
    return output_dir

# Fonction de sauvegarde des modèles (si mode ré-entraînement)
def backup_models(models_dict, backup_dir="models_backup"):
    """Sauvegarde de sécurité des modèles"""
    os.makedirs(backup_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    print(f"💾 Sauvegarde de sécurité des modèles dans: {backup_dir}/")
    
    for target_col, model_info in models_dict.items():
        # Sauvegarde du modèle
        model_backup_path = os.path.join(backup_dir, f"{target_col}_LSTM_backup_{timestamp}.h5")
        model_info['model'].save(model_backup_path)
        
        # Sauvegarde du scaler
        scaler_backup_path = os.path.join(backup_dir, f"{target_col}_scaler_backup_{timestamp}.pkl")
        with open(scaler_backup_path, 'wb') as f:
            pickle.dump(model_info['scaler'], f)
        
        print(f"   ✅ Modèle et scaler sauvés pour: {target_col}")

# Exécution de la sauvegarde
if 'all_predictions' in globals() and 'model_metrics' in globals():
    print("💾 Début de la sauvegarde des résultats...")
    
    # Sauvegarde des résultats
    results_dir = save_results(all_predictions, model_metrics)
    
    # Sauvegarde de sécurité des modèles si mode ré-entraînement
    if selected_mode == 'retrain' and 'trained_models' in globals():
        backup_models(trained_models)
    
    print(f"\\n🎉 Sauvegarde terminée avec succès!")
    print(f"📁 Tous les fichiers sont disponibles dans: {os.path.abspath(results_dir)}")
    
    # Résumé final
    print("\\n📋 RÉSUMÉ FINAL:")
    print("=" * 40)
    print(f"🎯 Mode utilisé: {selected_mode}")
    print(f"📊 Variables prédites: {len(all_predictions)}")
    print(f"📈 Prédictions futures: 30 jours par variable")
    
    # Affichage des meilleures performances
    best_model = max(model_metrics.items(), key=lambda x: x[1]['R²'])
    worst_model = min(model_metrics.items(), key=lambda x: x[1]['R²'])
    
    print(f"🏆 Meilleur modèle: {best_model[0]} (R² = {best_model[1]['R²']:.4f})")
    print(f"⚠️ Modèle à améliorer: {worst_model[0]} (R² = {worst_model[1]['R²']:.4f})")
    
    avg_r2 = np.mean([metrics['R²'] for metrics in model_metrics.values()])
    print(f"📊 R² moyen: {avg_r2:.4f}")
    
    if avg_r2 >= 0.8:
        print("✅ Performance globale: EXCELLENTE")
    elif avg_r2 >= 0.6:
        print("🟢 Performance globale: BONNE")
    else:
        print("🟡 Performance globale: À AMÉLIORER")

else:
    print("⚠️ Aucun résultat à sauvegarder. Exécutez d'abord les sections précédentes.")

## 🎯 8. Conclusion et Guide d'Utilisation

### ✅ Félicitations !

Vous avez terminé avec succès l'utilisation du système de prédiction énergétique LSTM !

### 📋 Récapitulatif des étapes accomplies :

1. **⚙️ Configuration** - Choix entre modèles pré-entraînés ou ré-entraînement
2. **📊 Données** - Chargement et préparation des données
3. **🤖 Modèles** - Utilisation ou création des modèles LSTM
4. **🔮 Prédictions** - Génération des prédictions futures (30 jours)
5. **📈 Évaluation** - Calcul des métriques de performance
6. **💾 Sauvegarde** - Export des résultats en CSV et rapport de synthèse

### 📁 Fichiers générés :

- **Prédictions futures** : `[variable]_predictions_futures_[timestamp].csv`
- **Évaluations** : `[variable]_evaluation_[timestamp].csv`  
- **Métriques** : `metriques_modeles_[timestamp].csv`
- **Rapport** : `rapport_synthese_[timestamp].txt`
- **Modèles sauvés** : Dossiers `models/` et `scalers/`

### 🔄 Pour de nouvelles prédictions :

1. **Nouvelles données** : Changez le fichier dans la section 2
2. **Nouveaux modèles** : Sélectionnez "Ré-entraîner" dans la section 2
3. **Paramètres** : Modifiez `SEQUENCE_LENGTH` et `PREDICTION_DAYS` dans la section 5

### 📞 Support :

Pour toute question ou amélioration, consultez la documentation du projet ou contactez l'équipe de développement.

---

**🔋 Merci d'utiliser le Système de Prédiction Énergétique LSTM !**

## 🤖 4. Entraînement des Modèles LSTM

Cette section entraîne les modèles LSTM pour chaque variable si le mode "retrain" est sélectionné.

In [ ]:
# Configuration pour l'entraînement
SEQUENCE_LENGTH = 30
EPOCHS = 50
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2

# Variables à prédire
variables_to_predict = [
    'prectotcorr', 'suface_pressure(pa)', 'temp2_ave(c)', 
    'temp2_max(c)', 'temp2_min(c)', 'total_demand(mw)',
    'wind_speed50_ave(ms)', 'wind_speed50_max(ms)', 'wind_speed50_min(ms)'
]

def create_lstm_model(input_shape):
    """Créer un modèle LSTM optimisé"""
    model = Sequential([
        LSTM(100, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25, activation='relu'),
        Dense(1)
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mean_squared_error',
        metrics=['mae']
    )
    
    return model

def prepare_sequences(data, sequence_length):
    """Préparer les séquences pour l'entraînement LSTM"""
    X, y = [], []
    for i in range(sequence_length, len(data)):
        X.append(data[i-sequence_length:i])
        y.append(data[i])
    return np.array(X), np.array(y)

# Dictionnaires pour stocker les modèles et scalers
trained_models = {}
trained_scalers = {}
training_history = {}

if user_mode == 'retrain':
    print("🔄 Début de l'entraînement des modèles personnalisés...")
    print("=" * 60)
    
    # Entraîner un modèle pour chaque variable
    for i, variable in enumerate(variables_to_predict):
        print(f"\n🤖 Entraînement du modèle pour : {variable}")
        print(f"📊 Progression : {i+1}/{len(variables_to_predict)}")
        
        if variable in df.columns:
            # Préparation des données
            data = df[variable].values.reshape(-1, 1)
            
            # Normalisation
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data)
            
            # Création des séquences
            X, y = prepare_sequences(scaled_data, SEQUENCE_LENGTH)
            
            if len(X) > 0:
                # Division train/test
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y, test_size=0.2, shuffle=False
                )
                
                # Création et entraînement du modèle
                model = create_lstm_model((SEQUENCE_LENGTH, 1))
                
                # Callbacks
                early_stopping = EarlyStopping(
                    monitor='val_loss', 
                    patience=10, 
                    restore_best_weights=True
                )
                
                model_checkpoint = ModelCheckpoint(
                    f'{MODELS_PATH}{variable}_LSTM.h5',
                    monitor='val_loss',
                    save_best_only=True
                )
                
                # Entraînement
                history = model.fit(
                    X_train, y_train,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_split=VALIDATION_SPLIT,
                    callbacks=[early_stopping, model_checkpoint],
                    verbose=1
                )
                
                # Sauvegarde
                trained_models[variable] = model
                trained_scalers[variable] = scaler
                training_history[variable] = history.history
                
                # Sauvegarde du scaler
                joblib.dump(scaler, f'{SCALERS_PATH}{variable}_scaler.pkl')
                
                # Évaluation rapide
                train_loss = model.evaluate(X_train, y_train, verbose=0)
                test_loss = model.evaluate(X_test, y_test, verbose=0)
                
                print(f"✅ {variable} - Train Loss: {train_loss[0]:.4f}, Test Loss: {test_loss[0]:.4f}")
            else:
                print(f"⚠️ Données insuffisantes pour {variable}")
        else:
            print(f"❌ Variable {variable} non trouvée dans les données")
    
    print("\n🎉 Entraînement terminé pour tous les modèles!")
    print(f"📁 Modèles sauvegardés dans : {MODELS_PATH}")
    print(f"📁 Scalers sauvegardés dans : {SCALERS_PATH}")

else:
    print("🎯 Utilisation des modèles pré-entraînés...")
    
    # Charger les modèles pré-entraînés
    for variable in variables_to_predict:
        model_path = f'{MODELS_PATH}{variable}_LSTM.h5'
        scaler_path = f'{SCALERS_PATH}{variable}_scaler.pkl'
        
        if os.path.exists(model_path) and os.path.exists(scaler_path):
            try:
                trained_models[variable] = load_model(model_path)
                trained_scalers[variable] = joblib.load(scaler_path)
                print(f"✅ Modèle chargé pour {variable}")
            except Exception as e:
                print(f"❌ Erreur lors du chargement de {variable}: {e}")
        else:
            print(f"⚠️ Modèle pré-entraîné non trouvé pour {variable}")

print(f"\n📊 Modèles disponibles : {len(trained_models)}")
print(f"🔧 Scalers disponibles : {len(trained_scalers)}")

In [ ]:
# Sauvegarde des résultats pour l'interface Streamlit
def save_results_for_streamlit(predictions_results, output_path="../results.csv"):
    """Sauvegarde les résultats dans le format attendu par Streamlit"""
    
    # Créer un DataFrame avec les résultats
    results_data = []
    
    # Générer les dates futures (30 jours à partir d'aujourd'hui)
    from datetime import datetime, timedelta
    base_date = datetime.now()
    
    # Prendre le premier modèle pour déterminer le nombre de prédictions
    first_key = list(predictions_results.keys())[0]
    num_predictions = len(predictions_results[first_key]['future_predictions'])
    
    for i in range(num_predictions):
        date = (base_date + timedelta(days=i+1)).strftime('%Y-%m-%d')
        
        # Initialiser le dictionnaire de résultats pour ce jour
        result_row = {'date': date}
        
        # Ajouter les prédictions pour chaque variable
        for variable, data in predictions_results.items():
            if i < len(data['future_predictions']):
                result_row[f'{variable}_predit'] = data['future_predictions'][i]
        
        # Calculer génération et consommation si les variables appropriées existent
        generation_vars = ['prectotcorr', 'wind_speed50_ave(ms)', 'wind_speed50_max(ms)']
        consumption_vars = ['total_demand(mw)', 'temp2_ave(c)', 'temp2_max(c)']
        
        # Estimation de la génération (basée sur les variables météo)
        generation = 0
        gen_count = 0
        for var in generation_vars:
            if f'{var}_predit' in result_row:
                # Normalisation et pondération pour estimation génération
                if 'wind' in var:
                    generation += result_row[f'{var}_predit'] * 10  # Facteur éolien
                    gen_count += 1
                elif 'prectot' in var:
                    generation += result_row[f'{var}_predit'] * 5   # Facteur hydroélectrique
                    gen_count += 1
        
        if gen_count > 0:
            generation = generation / gen_count
        else:
            generation = 100  # Valeur par défaut
        
        # Consommation (basée sur la demande totale ou température)
        consumption = 0
        cons_count = 0
        for var in consumption_vars:
            if f'{var}_predit' in result_row:
                if 'demand' in var:
                    consumption = result_row[f'{var}_predit']  # Demande directe
                    cons_count = 1
                    break
                elif 'temp' in var:
                    # Estimation basée sur la température
                    temp = result_row[f'{var}_predit']
                    consumption += abs(temp - 20) * 5  # Facteur de climatisation/chauffage
                    cons_count += 1
        
        if cons_count == 0:
            consumption = 80  # Valeur par défaut
        elif 'demand' not in [var for var in consumption_vars if f'{var}_predit' in result_row]:
            consumption = consumption / cons_count
        
        result_row['generation_predite'] = max(0, generation)
        result_row['consommation_predite'] = max(0, consumption)
        
        results_data.append(result_row)
    
    # Créer le DataFrame final
    results_df = pd.DataFrame(results_data)
    
    # Sauvegarder en CSV
    results_df.to_csv(output_path, index=False)
    print(f"✅ Résultats sauvegardés pour Streamlit : {output_path}")
    print(f"📊 {len(results_df)} jours de prédictions sauvegardées")
    
    # Afficher un aperçu
    print("\\n📋 Aperçu des résultats :")
    display(results_df.head())
    
    return results_df

# Exécuter la sauvegarde si des prédictions ont été générées
if 'all_predictions' in globals() and all_predictions:
    print("💾 Sauvegarde des résultats pour l'interface Streamlit...")
    streamlit_results = save_results_for_streamlit(all_predictions)
    print("✅ Sauvegarde terminée !")
else:
    print("⚠️ Aucune prédiction trouvée à sauvegarder")